<a href="https://colab.research.google.com/github/apal9569/How-severe-can-an-airplane-accident-be-/blob/master/HE_plane_severe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
cd drive/My\ Drive

/content/drive/My Drive


In [0]:
!unzip AIRPLANE_SEVERE.zip -d HE_plane

In [0]:
!rm AIRPLANE_SEVERE.zip

In [0]:
cd HE_plane/

In [0]:
import pandas as pd
import numpy as np


In [0]:
train=pd.read_csv('train.csv')

In [0]:
train.head(10)

In [27]:
train.nunique()

Severity                       4
Safety_Score                1175
Days_Since_Inspection         23
Total_Safety_Complaints       44
Control_Metric               952
Turbulence_In_gforces       1117
Cabin_Temperature            950
Accident_Type_Code             7
Max_Elevation               4191
Violations                     6
Adverse_Weather_Metric      4258
Accident_ID                10000
dtype: int64

In [0]:
label=train['Severity']
train=train.drop(columns=['Severity','Accident_ID'])

In [29]:
nan_col_before=train.columns[train.isna().any()].tolist()
nan_col_before

[]

In [0]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [0]:
train.describe()

,Safety_Score,Days_Since_Inspection,Total_Safety_Complaints,Control_Metric,Turbulence_In_gforces,Cabin_Temperature,Accident_Type_Code,Max_Elevation,Violations,Adverse_Weather_Metric
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000
mean,41.876406,12.931100,6.564300,65.145324,0.381495,79.969331,3.814900,32001.803282,2.01220,0.255635
std,16.138072,3.539803,6.971982,11.882934,0.121301,2.759739,1.902577,9431.995196,1.03998,0.381128
min,0.000000,1.000000,0.000000,0.000000,0.134000,74.740000,1.000000,831.695553,0.00000,0.000316
25%,30.593607,11.000000,2.000000,56.927985,0.293665,77.960000,2.000000,25757.636908,1.00000,0.012063
50%,41.278539,13.000000,4.000000,65.587967,0.365879,79.540000,4.000000,32060.336419,2.00000,0.074467
75%,52.511416,15.000000,9.000000,73.336372,0.451346,81.560000,5.000000,38380.641513,3.00000,0.354059
max,100.000000,23.000000,54.000000,100.000000,0.882648,97.510000,7.000000,64297.651218,5.00000,2.365378


In [0]:
le=LabelEncoder()
label=le.fit_transform(label)

In [12]:
from sklearn.preprocessing import MinMaxScaler
x=train['Safety_Score'].values
minmax=MinMaxScaler()
pd.DataFrame(minmax.fit_transform([x]).transpose()).nunique()

0    1
dtype: int64

In [0]:
data=(train-train.min())/(train.max()-train.min())

In [32]:
data.describe()

,Safety_Score,Days_Since_Inspection,Total_Safety_Complaints,Control_Metric,Turbulence_In_gforces,Cabin_Temperature,Accident_Type_Code,Max_Elevation,Violations,Adverse_Weather_Metric
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,0.418764,0.542323,0.121561,0.651453,0.330589,0.229659,0.469150,0.491131,0.402440,0.107954
std,0.161381,0.160900,0.129111,0.118829,0.162027,0.121201,0.317096,0.148615,0.207996,0.161149
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.305936,0.454545,0.037037,0.569280,0.213271,0.141414,0.166667,0.392745,0.200000,0.004967
50%,0.412785,0.545455,0.074074,0.655880,0.309730,0.210804,0.500000,0.492053,0.400000,0.031353
75%,0.525114,0.636364,0.166667,0.733364,0.423892,0.299517,0.666667,0.591639,0.600000,0.149570
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [15]:
from keras import layers
from keras.layers import *
from keras.models import Model
from keras.preprocessing.image import load_img,img_to_array
#from keras.applications.vgg19 import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.optimizers import RMSprop,Adam

Using TensorFlow backend.


In [0]:
from sklearn.preprocessing import OneHotEncoder
label=pd.DataFrame(label)
val=label.values
onehot_encoder = OneHotEncoder(sparse=False)
label=onehot_encoder.fit_transform(val)


In [0]:
trainx,testx,trainy,testy=train_test_split(data,label,test_size=0.15)

In [37]:
trainy

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       ...,
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.]])

In [0]:
inp=Input(shape=(train.shape[1],))
#den=Dense(256,activation='relu')(inp)
den=Dense(128,activation='relu')(inp)
den=Dense(128,activation='relu')(den)
den=Dense(64,activation='relu')(den)
den=Dense(64,activation='relu')(den)
den=Dense(32,activation='relu')(den)

den=Dense(label.shape[1],activation='softmax')(den)

model=Model(inputs=inp, outputs=den)


In [0]:
adam=Adam(lr=0.005,beta_1=0.95,beta_2=0.95,epsilon=0.001)
model.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['categorical_accuracy'])

In [111]:
history=model.fit(trainx,trainy,epochs=20,batch_size=256,verbose=1,validation_data=(testx,testy))

Train on 8500 samples, validate on 1500 samples
Epoch 1/20
8500/8500 [==============================] - 1s 115us/step - loss: 1.3182 - categorical_accuracy: 0.3479 - val_loss: 1.2577 - val_categorical_accuracy: 0.4560
Epoch 2/20
8500/8500 [==============================] - 0s 17us/step - loss: 1.1694 - categorical_accuracy: 0.4736 - val_loss: 1.0344 - val_categorical_accuracy: 0.5920
Epoch 3/20
8500/8500 [==============================] - 0s 18us/step - loss: 0.9656 - categorical_accuracy: 0.6159 - val_loss: 0.9433 - val_categorical_accuracy: 0.6020
Epoch 4/20
8500/8500 [==============================] - 0s 18us/step - loss: 0.7925 - categorical_accuracy: 0.6941 - val_loss: 0.7102 - val_categorical_accuracy: 0.7160
Epoch 5/20
8500/8500 [==============================] - 0s 18us/step - loss: 0.5574 - categorical_accuracy: 0.7945 - val_loss: 0.5376 - val_categorical_accuracy: 0.8007
Epoch 6/20
8500/8500 [==============================] - 0s 18us/step - loss: 0.4105 - categorical_accuracy

In [0]:
model=XGBClassifier(max_depth=6,learning_rate=0.15,gamma=0.15)
model.fit(trainx,trainy)

In [41]:
!ls

sample_submission.csv  test.csv  train.csv


In [0]:
test=pd.read_csv('test.csv')

In [113]:
test.head()

,Safety_Score,Days_Since_Inspection,Total_Safety_Complaints,Control_Metric,Turbulence_In_gforces,Cabin_Temperature,Accident_Type_Code,Max_Elevation,Violations,Adverse_Weather_Metric,Accident_ID
0,19.497717,16,6,72.151322,0.388959,78.32,4,37949.724386,2,0.069692,1
1,58.173516,15,3,64.585232,0.250841,78.60,7,30194.805567,2,0.002777,10
2,33.287671,15,3,64.721969,0.336669,86.96,6,17572.925484,1,0.004316,14
3,3.287671,21,5,66.362808,0.421775,80.86,3,40209.186341,2,0.199990,17
4,10.867580,18,2,56.107566,0.313228,79.22,2,35495.525408,2,0.483696,21


In [0]:
acc_id=test['Accident_ID']
test=test.drop(columns=['Accident_ID'])
data=(test-test.min())/(test.max()-test.min())

In [115]:
data.head()

,Safety_Score,Days_Since_Inspection,Total_Safety_Complaints,Control_Metric,Turbulence_In_gforces,Cabin_Temperature,Accident_Type_Code,Max_Elevation,Violations,Adverse_Weather_Metric
0,0.194977,0.681818,0.111111,0.664497,0.332520,0.183967,0.500000,0.603705,0.4,0.029313
1,0.581735,0.636364,0.055556,0.566272,0.145508,0.198356,1.000000,0.477575,0.4,0.001019
2,0.332877,0.636364,0.055556,0.568047,0.261719,0.627955,0.833333,0.272287,0.2,0.001670
3,0.032877,0.909091,0.092593,0.589349,0.376953,0.314491,0.333333,0.640454,0.4,0.084407
4,0.108676,0.772727,0.037037,0.456213,0.229980,0.230216,0.166667,0.563789,0.4,0.204366


In [116]:
prediction=model.predict(data)
prediction

array([[9.99849081e-01, 1.00369414e-10, 1.33317706e-04, 1.74914985e-05],
       [1.37761673e-02, 8.51018180e-04, 7.08662987e-01, 2.76709855e-01],
       [5.48491906e-03, 4.22260106e-01, 2.64209259e-04, 5.71990728e-01],
       ...,
       [1.51408883e-02, 3.00474085e-07, 2.99983016e-09, 9.84858811e-01],
       [8.50482285e-03, 3.61688551e-04, 4.97055737e-07, 9.91133034e-01],
       [1.24456927e-01, 1.78827599e-04, 8.75242710e-01, 1.21532437e-04]],
      dtype=float32)

In [0]:
prediction=prediction.argmax(axis=1)

In [118]:
prediction.shape,test.shape

((2500,), (2500, 10))

In [119]:
le.classes_

array(['Highly_Fatal_And_Damaging', 'Minor_Damage_And_Injuries',
       'Significant_Damage_And_Fatalities',
       'Significant_Damage_And_Serious_Injuries'], dtype=object)

In [0]:
out=le.inverse_transform(prediction)

In [121]:
out

array(['Highly_Fatal_And_Damaging', 'Significant_Damage_And_Fatalities',
       'Significant_Damage_And_Serious_Injuries', ...,
       'Significant_Damage_And_Serious_Injuries',
       'Significant_Damage_And_Serious_Injuries',
       'Significant_Damage_And_Fatalities'], dtype=object)

In [0]:
sub=pd.read_csv('sample_submission.csv')

In [123]:
sub.head()

,Accident_ID,Severity
0,1,Minor_Damage_And_Injuries
1,10,Highly_Fatal_And_Damaging
2,14,Highly_Fatal_And_Damaging
3,17,Significant_Damage_And_Serious_Injuries
4,21,Minor_Damage_And_Injuries


In [0]:
submission=pd.DataFrame(columns=['Accident_ID','Severity'])

In [125]:
submission

,Accident_ID,Severity


In [0]:
submission['Accident_ID']=acc_id
submission['Severity']=out

In [0]:
submission.to_csv('Prediction.csv',index=False)